In [1]:
import psycopg2
import os

First lets connect ourselves with our POSTGRES db. The credentials are also copied in the `docker-compose.yml`.
So before you are exec the next cell, please make sure that docker is installed on your machine. :)

In [3]:
# Connect to the database by exec docker compose in your terminal. This exec. a terminal command using Python
os.system("""
docker compose down;
docker compose up -d --build db
""")

#0 building with "desktop-linux" instance using docker driver

#1 [db internal] load build definition from Dockerfile
#1 transferring dockerfile: 159B done
#1 DONE 0.0s

#2 [db internal] load metadata for docker.io/library/postgres:latest
#2 DONE 0.0s

#3 [db internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [db 1/2] FROM docker.io/library/postgres:latest
#4 DONE 0.0s

#5 [db internal] load build context
#5 transferring context: 2.53kB done
#5 DONE 0.0s

#6 [db 2/2] COPY dump.sql /docker-entrypoint-initdb.d/
#6 CACHED

#7 [db] exporting to image
#7 exporting layers done
#7 writing image sha256:eb5d4bd74aef4c3760c9017d6061661e17b8cec7d10f95fa7e56ff2065a48544 done
#7 naming to docker.io/library/project_mse-db done
#7 DONE 0.0s


 Network project_mse_default  Creating
 Network project_mse_default  Created
 Container project_mse-db-1  Creating
 Container project_mse-db-1  Created
 Container project_mse-db-1  Starting
 Container project_mse-db-1  Started


0

When the cell returns 0, it means that everything was done correctly.

In [4]:
try:
    # Connect to your PostgreSQL database. Here are the credentials written so the machine knows where connect to.
    connection = psycopg2.connect(
        user="user",
        password="user_pw",
        host="localhost",
        port="5432",
        database="search_engine_db"
    )

    # Create a cursor object. We need this to execute queries.
    cursor = connection.cursor()

    # Print PostgreSQL version just as a sanity check.
    cursor.execute("SELECT version();")
    record = cursor.fetchone()
    print("You are connected to - ", record, "\n")

    # Example query: Create a table
    create_search_index_table = '''
        CREATE TABLE IF NOT EXISTS documents (
        id SERIAL PRIMARY KEY,
        url TEXT NOT NULL,
        keyword TEXT,
        content TEXT,
        last_updated TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );
    '''
    
    # Executes a query statement
    cursor.execute(create_search_index_table)
    
    # Commit the transaction to save changes. Should always be called after `cursor.execute(...)`
    connection.commit()
    print("Table created successfully")

    # Example query: Insert data
    insert_query_example = "INSERT INTO documents (url, content) VALUES (%s, %s)"
    
    cursor.execute(insert_query_example, ('testurl', 'This is a nice content page!'))
    connection.commit()
    print("Data inserted successfully")

    # Example query: Retrieve data
    cursor.execute("SELECT * FROM documents")
    records = cursor.fetchall()
    print("Data retrieved successfully:")
    for row in records:
        print(row)
    
    # To finish the connection with our db after you interact with it.
    cursor.close()
    connection.close()
    print("PostgreSQL connection is closed")

except (Exception, psycopg2.Error) as error:
        print("Error while connecting to PostgreSQL", error)

You are connected to -  ('PostgreSQL 16.3 (Debian 16.3-1.pgdg120+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit',) 

Table created successfully
Data inserted successfully
Data retrieved successfully:
(1, 'testurl', None, 'This is a nice content page!', datetime.datetime(2024, 6, 27, 18, 13, 19, 949041))
(2, 'testurl', None, 'This is a nice content page!', datetime.datetime(2024, 6, 27, 18, 13, 22, 702375))
(3, 'testurl', None, 'This is a nice content page!', datetime.datetime(2024, 6, 27, 18, 13, 24, 384018))
(4, 'testurl', None, 'This is a nice content page!', datetime.datetime(2024, 6, 27, 18, 13, 26, 161110))
(5, 'testurl', None, 'This is a nice content page!', datetime.datetime(2024, 6, 27, 18, 13, 28, 633620))
(6, 'testurl', None, 'This is a nice content page!', datetime.datetime(2024, 6, 27, 18, 30, 6, 63735))
PostgreSQL connection is closed


When you see `PostgreSQL connection is closed`, you know that everything was done correctly. Now you can peak into the database, with e.g. DBeaver and see how the data looks like.

## CAUTION
exec this table to delete all entries from the database


In [5]:
# CAUTION: exec this table to delete all entries from the database

try:
    # Connect to your PostgreSQL database. Here are the credentials written so the machine knows where connect to.
    connection = psycopg2.connect(
        user="user",
        password="user_pw",
        host="localhost",
        port="5432",
        database="search_engine_db"
    )

    # Create a cursor object. We need this to execute queries.
    cursor = connection.cursor()

    # Delete all entries
    cursor.execute("DELETE FROM documents")
    connection.commit()
    print("Entries deleted successfully")
    
    cursor.execute("SELECT * FROM documents")
    records = cursor.fetchall()
    print("Data retrieved successfully:")
    if len(records) == 0:
        print("No entries found")
    
    # To finish the connection with our db after you interact with it.
    cursor.close()
    connection.close()
    print("PostgreSQL connection is closed")

except (Exception, psycopg2.Error) as error:
        print("Error while connecting to PostgreSQL", error)

Entries deleted successfully
Data retrieved successfully:
No entries found
PostgreSQL connection is closed


Finally, you should shut down the container and kill it.

In [ ]:
os.system("""
docker compose down;
""")